# F1 Red Bull Racing Analytics

Analysis of Red Bull Racing's performance using the local SQLite database.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

# Connect to SQLite database
db_path = '../f1_analytics.db'
engine = create_engine(f'sqlite:///{db_path}')

# Set plot style
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')

## 1. Team Performance Overview
Win rate and podium statistics.

In [ ]:
query = """
SELECT 
    'Total Races' AS metric,
    COUNT(*) AS value
FROM results 
WHERE constructor_id = 9
UNION ALL
SELECT 'Total Wins', COUNT(*) FROM results WHERE constructor_id = 9 AND position = 1
UNION ALL
SELECT 'Total Podiums', COUNT(*) FROM results WHERE constructor_id = 9 AND position <= 3
"""

df_kpi = pd.read_sql(query, engine)
print(df_kpi)

# Calculate percentages
total_races = df_kpi[df_kpi['metric'] == 'Total Races']['value'].values[0]
wins = df_kpi[df_kpi['metric'] == 'Total Wins']['value'].values[0]
podiums = df_kpi[df_kpi['metric'] == 'Total Podiums']['value'].values[0]

print(f"\nWin Rate: {wins/total_races*100:.1f}%")
print(f"Podium Rate: {podiums/total_races*100:.1f}%")

## 2. Season Performance
Points scored per season.

In [ ]:
query = """
SELECT 
    r.year,
    SUM(res.points) as total_points,
    COUNT(CASE WHEN res.position = 1 THEN 1 END) as wins
FROM results res
JOIN races r ON res.race_id = r.race_id
WHERE res.constructor_id = 9
GROUP BY r.year
ORDER BY r.year
"""

df_season = pd.read_sql(query, engine)

plt.figure(figsize=(12, 6))
sns.barplot(data=df_season, x='year', y='total_points', color='navy')
plt.title('Red Bull Racing - Points per Season')
plt.xticks(rotation=45)
plt.ylabel('Points')
plt.show()

## 3. Driver Comparison
Who scored the most points for Red Bull?

In [ ]:
query = """
SELECT 
    d.forename || ' ' || d.surname as driver_name,
    SUM(res.points) as total_points,
    COUNT(CASE WHEN res.position = 1 THEN 1 END) as wins
FROM results res
JOIN drivers d ON res.driver_id = d.driver_id
WHERE res.constructor_id = 9
GROUP BY driver_name
ORDER BY total_points DESC
LIMIT 10
"""

df_drivers = pd.read_sql(query, engine)

plt.figure(figsize=(10, 6))
sns.barplot(data=df_drivers, y='driver_name', x='total_points', hue='driver_name', legend=False)
plt.title('Top Red Bull Drivers by Points')
plt.xlabel('Total Points')
plt.ylabel('')
plt.show()